In [55]:
import numpy as np 
import torch
import torch.nn as nn

In [56]:
from google.colab import drive # 連接到google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


匯入labels.csv

In [57]:
path = './drive/MyDrive/Dog/labels.csv' 
dogs_numpy = np.loadtxt(path, dtype=str, delimiter=',', skiprows=1) # 處理 excel有內建分隔delimiter 去除第一行(id breed)
dogs_numpy, dogs_numpy.shape

(array([['000bec180eb18c7604dcecc8fe0dba07', 'boston_bull'],
        ['001513dfcb2ffafc82cccf4d8bbaba97', 'dingo'],
        ['001cdf01b096e06d78e9e5112d419397', 'pekinese'],
        ...,
        ['ffe2ca6c940cddfee68fa3cc6c63213f', 'airedale'],
        ['ffe5f6d8e2bff356e9482a80a6e29aac', 'miniature_pinscher'],
        ['fff43b07992508bc822f33d8ffd902ae', 'chesapeake_bay_retriever']],
       dtype='<U32'), (10222, 2))

匯入breed.txt

In [58]:
path = './drive/MyDrive/Dog/breed.txt' 
breed_numpy = np.loadtxt(path, dtype=str)
#breed_numpy

In [59]:
# 轉成list
breed_list = list() # (可略) 空list為list()或[]
breed_list = list(breed_numpy)
#breed_list

In [60]:
# 轉成dic
dic = {} # (不可略) 空dict為dict()或{}
for i in range(len(breed_list)):
    dic[breed_list[i]] = i # dic[ key ] = value
breed_numpy

array(['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale',
       'american_staffordshire_terrier', 'appenzeller',
       'australian_terrier', 'basenji', 'basset', 'beagle',
       'bedlington_terrier', 'bernese_mountain_dog',
       'black-and-tan_coonhound', 'blenheim_spaniel', 'bloodhound',
       'bluetick', 'border_collie', 'border_terrier', 'borzoi',
       'boston_bull', 'bouvier_des_flandres', 'boxer',
       'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff',
       'cairn', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua',
       'chow', 'clumber', 'cocker_spaniel', 'collie',
       'curly-coated_retriever', 'dandie_dinmont', 'dhole', 'dingo',
       'doberman', 'english_foxhound', 'english_setter',
       'english_springer', 'entlebucher', 'eskimo_dog',
       'flat-coated_retriever', 'french_bulldog', 'german_shepherd',
       'german_short-haired_pointer', 'giant_schnauzer',
       'golden_retriever', 'gordon_setter', 'great_dane',
      

處理dog_numpy(labels.csv)

In [61]:
dog_labels = list(dogs_numpy[:,-1]) # dogs_numpy為(10222, 2) 10222總狗 橫向看為(id, breed) [:,-1]為取所有狗的breed而已
dog_labels = [dic[breed_name] for breed_name in dog_labels] # dic[breed_name]最後會產出value 即找dog_labels內對應到dict的標號
dog_labels
dog_t_labels = torch.tensor(dog_labels) # 轉成tensor
dog_t_labels

# 舉例: dogs_numpy的第一種狗'boston_bull' 在breed.txt中為第19個 故dic['boston_bull']=19 而dog_labels[0]和dog_t_labels[0]為19和tensor(19)

tensor([19, 37, 85,  ...,  3, 75, 28])

匯入train圖片

In [62]:
import os
dog_file_name = [i +'.jpg' for i in dogs_numpy[:,0]] # dogs_numpy[:,0]為id
dog_file_path = [os.path.join('./drive/MyDrive/Dog/train', i_jpg) # 取train中每一個圖片 os.path.join為合併路徑
          for i_jpg in dog_file_name] 
#dog_file_path

正規化

對一張圖片

In [63]:
from torchvision import transforms
# 資料集的圖片都不一樣大小 故需要先把圖片變成一樣大 後來轉成tensor才可以算mean和std
preprocess_resize_tensor = transforms.Compose([transforms.Resize([100, 100]), # 變正方形
                  transforms.ToTensor() # 轉tensor
])

from PIL import Image # 匯入外部圖片
img_real = Image.open(dog_file_path[1]) # 用第一張圖片作為試驗 為真正的圖片
img_tensor = preprocess_resize_tensor(img_real) # 處理圖片
img_tensor # torch.Size([3, 224, 224])

# 保留三個RGB channels 剩下的皆合併 一張圖片的shape變成torch.Size([3, 50176]) 最後對三個不同的channel求mean和std
img_tensor.view(3, -1).mean(dim=1), img_tensor.view(3, -1).std(dim=1) 

(tensor([0.4658, 0.3058, 0.1603]), tensor([0.2985, 0.2460, 0.1410]))

對所有資料集

In [64]:
"""
means = torch.zeros(3)
bias = torch.zeros(3)
means1 = torch.zeros(3)
bias1 = torch.zeros(3)
means2 = torch.zeros(3)
bias2 = torch.zeros(3)


for i in range (0, 4000):
  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片
  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片
  means += imgs_tensor.view(3, -1).mean(dim=1)
  bias += imgs_tensor.view(3, -1).std(dim=1)


for i in range (4000, 8000):
  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片
  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片
  means1 += imgs_tensor.view(3, -1).mean(dim=1)
  bias1 += imgs_tensor.view(3, -1).std(dim=1)

for i in range (8000, 10222):
  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片
  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片
  means2 += imgs_tensor.view(3, -1).mean(dim=1)
  bias2 += imgs_tensor.view(3, -1).std(dim=1)

(means + means1 + means2)/10222, (bias + bias1 + bias2)/10222

"""

'\nmeans = torch.zeros(3)\nbias = torch.zeros(3)\nmeans1 = torch.zeros(3)\nbias1 = torch.zeros(3)\nmeans2 = torch.zeros(3)\nbias2 = torch.zeros(3)\n\n\nfor i in range (0, 4000):\n  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片\n  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片\n  means += imgs_tensor.view(3, -1).mean(dim=1)\n  bias += imgs_tensor.view(3, -1).std(dim=1)\n\n\nfor i in range (4000, 8000):\n  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片\n  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片\n  means1 += imgs_tensor.view(3, -1).mean(dim=1)\n  bias1 += imgs_tensor.view(3, -1).std(dim=1)\n\nfor i in range (8000, 10222):\n  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片\n  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片\n  means2 += imgs_tensor.view(3, -1).mean(dim=1)\n  bias2 += imgs_tensor.view(3, -1).std(dim=1)\n\n(means + means1 + means2)/10222, (bias + bias1 + bias2)/10222\n\n'

In [71]:
means = torch.zeros(3)
bias = torch.zeros(3)

for i in range (0, 10222):
  imgs_real = Image.open(dog_file_path[i]) # 用第一張圖片作為試驗 為真正的圖片
  imgs_tensor = preprocess_resize_tensor(imgs_real) # 處理圖片
  means += imgs_tensor.view(3, -1).mean(dim=1)
  bias += imgs_tensor.view(3, -1).std(dim=1)

means/10222, bias/10222

(tensor([0.4766, 0.4523, 0.3923]), tensor([0.2205, 0.2154, 0.2137]))

前處理preprocess(含正規化)

In [72]:
from torchvision import transforms, utils

preprocess = transforms.Compose([transforms.Resize([224, 224]), # 變正方形
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.4766, 0.4523, 0.3923], std=[0.2205, 0.2154, 0.2137]) # 正規化
])

loader

In [73]:
from PIL import Image # 匯入外部圖片
def default_loader(path):
    img_pil = Image.open(path) # 打開真正的圖片
    img_tensor = preprocess(img_pil) # 處理圖片
    return img_tensor

定義資料集

In [74]:
# 定義類別 
from torch.utils.data import Dataset, DataLoader

class TrainSet(Dataset):
  def __init__(self, load=default_loader): # 初始化
      self.images = dog_file_path # dog_file_path為照片的完整路徑
      self.target = dog_t_labels # 真正的狗的labels
      self.loader = default_loader
  
  def __getitem__(self, index): # 內建getitem的規則(有點像是打開一張圖片的方法)
      fn = self.images[index] # 也就是dog_file_path[1]
      img = self.loader(fn) # 輸出真正的圖片 並前處理後 返回其tensor
      target = self.target[index] # dog_t_labels[1] 真正的labels
      return img, target

  def __len__(self):
    return len(self.images) # 資料集長度

In [75]:
train_data = TrainSet() # 把model改名

In [76]:
trainloader = DataLoader(train_data, batch_size=4, shuffle=True) # DataLoader
for imgs, labels in trainloader:
  batch_size = imgs.shape[0]
  batch_size
  break
imgs, labels

(tensor([[[[-0.2407, -0.2051, -0.2051,  ...,  2.0536,  2.0358,  2.0180],
           [-0.1873, -0.1162, -0.1695,  ...,  2.0714,  2.0536,  2.0358],
           [-0.1162, -0.1162, -0.0984,  ...,  2.0180,  2.0002,  1.9824],
           ...,
           [ 0.0972,  0.1328,  0.0083,  ..., -1.3967, -1.3789, -1.7702],
           [ 0.0439,  0.0261,  0.0261,  ..., -1.6101, -1.6813, -1.6457],
           [ 0.0794,  0.0794,  0.1328,  ..., -1.5212, -1.5390, -1.6101]],
 
          [[-0.3885, -0.3520, -0.3520,  ...,  2.1058,  2.0876,  2.0694],
           [-0.3338, -0.2610, -0.3156,  ...,  2.1240,  2.1058,  2.0876],
           [-0.2610, -0.2610, -0.2428,  ...,  2.0694,  2.0511,  2.0329],
           ...,
           [-0.0790, -0.0425, -0.1700,  ..., -1.4262, -1.4080, -1.8085],
           [-0.1336, -0.1518, -0.1336,  ..., -1.6811, -1.7539, -1.7175],
           [-0.0972, -0.0972, -0.0243,  ..., -1.6083, -1.6265, -1.6993]],
 
          [[-0.3493, -0.2943, -0.2759,  ...,  2.4400,  2.4216,  2.4033],
           [-